# Pytorch Convoltion Layer 뜯어보기

In [1]:
import torch.optim as optim
import torch.nn as nn
import torch

입력을 위한 간단한 데이터를 생성해줍니다. 데이터 생성은 `torch.randn()` 함수를 통해 생성합니다. `torch.randn()` 함수는 아래와 같이 생성됩니다. 들어가는 입력값은 모두 랜덤으로 들어가게 됩니다.

### 1차원 행렬 생성

`torch.randn(2)` 라고 했을 경우에는 $2$ 만큼의 크기를 지닌 1차원 행렬을 생성하게 됩니다.

### 2차원 행렬 생성

만일 `torch.randn(2, 4)` 로 했으면 $2*4$ 크기의 2차원 행렬을 생성하게 됩니다. 반대로 `torch.randn(4, 2)` 로 할 경우에는 $4*2$ 행렬을 생성합니다.

---

위와 같은 방법으로 N차원의 행렬을 생성할 수 있으나 더욱이 많은 차원과 데이터가 들어갈수록 메모리 할당에 부담을 줄 수 있으니 주의해야합니다. 

In [33]:
input_data = torch.randn(3, 3)
print(input_data)
print(input_data.size())

tensor([[-0.6212,  1.0248, -1.0840],
        [ 2.1332, -0.0533, -0.8639],
        [-0.3711,  0.0931, -0.4796]])
torch.Size([3, 3])


# nn.Conv1d 작동 방법

(Pytorch 공식문서)[https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#torch.nn.Conv1d] 에 따르면 `nn.Conv1d()` 함수에 들어가는 입력값은 아래와 같습니다.

```
in_channels (int) – Number of channels in the input image
out_channels (int) – Number of channels produced by the convolution
kernel_size (int or tuple) – Size of the convolving kernel
stride (int or tuple, optional) – Stride of the convolution. Default: 1
padding (int, tuple or str, optional) – Padding added to both sides of the input. Default: 0
padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'
dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1
groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1
bias (bool, optional) – If True, adds a learnable bias to the output. Default: True
```

주로 쓰이게 되고 주의 해야하는 것은 `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding` 입니다.

## in_channels

`in_channels`는 들어가게 될 입력값의 채널을 말합니다. 입력된 데이터에 형태에 따라 다릅니다. $3*3$ 크기의 행렬을 `torch.randn(3, 3)` 으로 생성했다고 가정하겠습니다. 생성하게 되면 크기는 `torch.Size([3, 3])` 으로 나오게 됩니다. 행렬의 차원은 1-Dimension 이지만, 입력을 받게 되면 첫번째 $3$은 채널 수를, 두번째 $3$은 입력값의 길이를 나타납니다. 지금 같은 경우에는 batchsize가 따로 정해지지 않았지만 만일 batch도 지정하게 된다면 $(Batchsize * Channel_in, * Channel_out)$ 으로 나오게 됩니다.

## out_channels

`out_channels`는 모델이 뱉어내게 될 행렬의 크기를 말합니다. 만일 `in_channels=3`으로 지정하고 `out_channels=4` 라고 한다면 기존 행렬보다 큰 크기로 결과를 배출하게 됩니다. 반드시 `out_channels` 의 크기에 따라 결과값이 변하는 것은 아닙니다. 물론 영향을 주지만 다음에 얘기할 `kernel_size`, `stride` 에 따라서도 결과값의 크기를 달라집니다.

## kernel_size

CNN (Convolutional Neural Network) 에 대한 내용은 생략하겠습니다. 무튼, Convolution Layer 에서 쓰이는 $Kernel Size$ 는 Pytorch Library 에서도 똑같은 역할을 합니다. 

> 얼마만큼의 크기로 주어진 데이터를 확인할 것인가?
> 얼마나 작게 작게 볼 것인가?

등을 결정합니다. $3*3$ 행렬이 있다고 가정할 떄, 이를 2개씩 보고 연산을 할 것인가를 결정하게 됩니다.

## stride

Stride 은 Kernel size을 몇칸 씩 이동시킬 것인지에 대해 정의하게 됩니다. 이때 __한 칸__ 이라고 불리는 기준은 한 원소라고 보면 됩니다. $3*3$ 행렬에서 $kernel_size=2$ 인 모델을 $stride=2$ 로 보게 된다면 2 크기만큼의 convolutional 한 kernel를 2칸씩 통과시켜 연산한다는 의미를 지닙니다.

In [64]:
in_channel = 3
out_channel = 3
kernel_size_list = [1, 2, 3]
strides_list = [1, 2, 3]

for kernel_val, stride_val in zip(kernel_size_list, strides_list):
    temp_model = nn.Conv1d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_val, stride=stride_val)
    print(f"[Kernel Size : {kernel_val}, Stride : {stride_val}]\nOutput size : \n{temp_model(input_data)}")

[Kernel Size : 1, Stride : 1]
Output size : 
tensor([[ 0.7813, -0.5015,  0.3432],
        [-0.0542, -0.1079,  1.6483],
        [ 0.3992, -0.1869, -0.3700]], grad_fn=<SqueezeBackward1>)
[Kernel Size : 2, Stride : 2]
Output size : 
tensor([[-0.2740],
        [ 0.1224],
        [ 0.6526]], grad_fn=<SqueezeBackward1>)
[Kernel Size : 3, Stride : 3]
Output size : 
tensor([[-1.0784],
        [-0.6802],
        [-0.5718]], grad_fn=<SqueezeBackward1>)


때문에 위와 같이 out_channel, kernel size, stride 에 의해서 결과값이 달라지기에 이는 중요한 Parameter라고 할 수 있다. 데이터를 얼마나 보냐, 데이터를 어떻게 처리 하냐에 따라 결과값이 달라지게 된다. 더불어 연속적인 모델링을 할 때에는 더욱의 크기에 의해 주의 해야한다.